# Linked Brushing Demo

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/anitagraser/movingpandas-examples/main?filepath=3-tech-demos/linked-brushing.ipynb)

This notebook demonstrates linked brushing with **Holoviews.selection**.


In [ ]:
from datetime import datetime
import numpy as np
import pandas as pd
import geopandas as gpd
import holoviews as hv
import hvplot.pandas  # noqa
from datashader.utils import lnglat_to_meters
from holoviews.element import tiles
from holoviews.util.transform import dim
from holoviews.selection import link_selections
from holoviews.operation import gridmatrix
from holoviews.operation.element import histogram
from holoviews import opts

import warnings
warnings.simplefilter("ignore")

In [ ]:
hv.__version__

In [ ]:
opts.defaults(opts.Overlay(active_tools=['wheel_zoom']))

In [ ]:
gdf = gpd.read_file('../data/ais.gpkg', rows=1000)
gdf.head()

In [ ]:
gdf.loc[:, 'x'], gdf.loc[:, 'y'] = lnglat_to_meters(gdf.geometry.x, gdf.geometry.y)
df = pd.DataFrame(gdf).head(10000)

## Pandas DataFrame.hvplot

In [ ]:
hist_plot = df.where((df.SOG>0) & (df.SOG<50)).hvplot.hist("SOG",  bins=20, width=400, height=300) 
map_plot = df.hvplot.scatter(x='x', y='y', width=400, height=300)
link_selections(tiles.CartoLight() * map_plot + hist_plot)

In [ ]:
bar_plot = df.groupby("ShipType").count().hvplot.bar(x="ShipType", y='MMSI', width=400, rot=90)
map_plot + bar_plot

In [ ]:
def selected_bar_plot(x_range, y_range):
    if x_range:
        tmp = df[(df.x > x_range[0]) & (df.x < x_range[1]) & (df.y > y_range[0]) & (df.y < y_range[1])]
    else:
        tmp = df
    counts = tmp.groupby("ShipType").count()
    counts = counts.reset_index()
    n_all = counts.MMSI.sum()
    n_max = counts.MMSI.max()
    return counts.hvplot.bar(title=f"{n_max} records", x="ShipType", y='MMSI', width=400, height=400, rot=90, color="ShipType", colormap="Category20", ylim=(0,n_max),)

map_plot = df.hvplot.scatter(title=f"AIS", x='x', y='y', c="ShipType", width=700, height=400, colormap="Category20")
rangexy = hv.streams.RangeXY(source=map_plot)
    
map_plot + hv.DynamicMap(selected_bar_plot, streams=[rangexy])

In [ ]:
map_plot << hv.DynamicMap(selected_bar_plot, streams=[rangexy])

In [ ]:
map_plot + hv.DynamicMap(selected_bar_plot, streams=[rangexy])

## Geopandas GeoDataFrame.hvplot

To use the GeoDataFrame directly, we need to explicitly set a suitable index for linking, as described in https://github.com/holoviz/geoviews/issues/497

In [ ]:
gdf['id'] = np.arange(len(gdf))
gdf_map = gdf.hvplot(geo=True, tiles='CartoLight', width=400, height=300)
gdf_hist = pd.DataFrame(gdf).where((gdf.SOG>0) & (gdf.SOG<50)).hvplot.hist("SOG",  bins=20, width=400, height=300)
link_selections(gdf_map + gdf_hist, index_cols=['id'])

## Datashade

In [ ]:
gdf = gpd.read_file('../data/ais.gpkg', rows=100000)
gdf.loc[:, 'x'], gdf.loc[:, 'y'] = lnglat_to_meters(gdf.geometry.x, gdf.geometry.y)
df = pd.DataFrame(gdf)

In [ ]:
hist_plot = df.where((df.SOG>0) & (df.SOG<50)).hvplot.hist("SOG", bins=20, width=400, height=300) 

In [ ]:
datashade = df.hvplot.scatter(x='x', y='y', datashade=True, width=400, height=300)
link_selections(datashade + hist_plot)#.cols(1)

In [ ]:
link_selections(tiles.CartoLight() * datashade + hist_plot)#.cols(1)

In [ ]:
df

## Bar plots (unsupported)

It would be nice to add a bar plot with counts per ship type but bar plots are currently not supported, see http://holoviews.org/user_guide/Linked_Brushing.html

In [ ]:
bar_plot = df.groupby('ShipType').agg({'SOG':'count'}).rename(columns={'SOG':'count'}).hvplot.barh(width=400, height=400)
bar_plot